# Project Description:

The purpose of this project is to implement a neural network that performs the translation of mathematical formulae from traditional **infix notation**—where the operator appears between two operands—to **postfix** (also known as Reverse Polish Notation), where the operator follows the operands.

Infix notation is the most commonly used in human-readable mathematics (e.g., a + b), but it is inherently ambiguous without additional syntactic aids such as parentheses or operator precedence rules. This ambiguity arises because different parse trees can correspond to the same expression depending on how operations are grouped.

In contrast, postfix notation eliminates the need for parentheses entirely. The order of operations is explicitly encoded by the position of the operators relative to the operands, making it more suitable for stack-based evaluation and easier to parse programmatically.

**Example:**

Consider the ambiguous infix expression:
a + b * c

This expression can be parsed in at least two different ways:

Interpretation (Infix):	(a + b) * c	   
Equivalent Postfix: ab+c*

Interpretation (Infix):	a + (b * c)	          
Equivalent Postfix: abc*+


This project aims to learn such disambiguations and generate the correct postfix form from a given infix expression using a data-driven approach based on neural networks. To simplify the task and control the complexity of expressions, we restrict our dataset to formulae with a maximum syntactic depth of 3. This means that the abstract syntax trees representing these expressions will have at most three levels, ensuring that the neural network operates on a bounded and manageable set of possible structures.

In [1]:
import numpy as np
import random
import string
import tensorflow as tf
from tensorflow.keras import layers, models

2025-06-11 12:10:44.094238: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 12:10:44.099522: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-11 12:10:44.115403: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749636644.143271    7303 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749636644.150632    7303 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749636644.170206    7303 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

We build formulae using 5 identifiers a,b,c,d,e and 4 binary operators +,-,*,/.
For simplicity we do not take advantage of precedence or associativity rules for infix notation, and suppose that all binary operations as always fully parenthesizes: (e1 op e2).

In [2]:
# -------------------- Constants --------------------
OPERATORS = ['+', '-', '*', '/']
IDENTIFIERS = list('abcde')
SPECIAL_TOKENS = ['PAD', 'SOS', 'EOS']
SYMBOLS = ['(', ')', '+', '-', '*', '/']
VOCAB = SPECIAL_TOKENS + SYMBOLS + IDENTIFIERS + ['JUNK'] #may use junk in autoregressive generation

token_to_id = {tok: i for i, tok in enumerate(VOCAB)}
id_to_token = {i: tok for tok, i in token_to_id.items()}
VOCAB_SIZE = len(VOCAB)
PAD_ID = token_to_id['PAD']
EOS_ID = token_to_id['EOS']
SOS_ID = token_to_id['SOS']

MAX_DEPTH = 3
MAX_LEN = 4*2**MAX_DEPTH -2 #enough to fit expressions at given depth (+ EOS)

In [3]:
# -------------------- Expression Generation --------------------
def generate_infix_expression(max_depth):
    if max_depth == 0:
        return random.choice(IDENTIFIERS)
    elif random.random() < 0.5:
        return generate_infix_expression(max_depth - 1)
    else:
        left = generate_infix_expression(max_depth - 1)
        right = generate_infix_expression(max_depth - 1)
        op = random.choice(OPERATORS)
        return f'({left} {op} {right})'

def tokenize(expr):
    return [c for c in expr if c in token_to_id]

def infix_to_postfix(tokens):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2}
    output, stack = [], []
    for token in tokens:
        if token in IDENTIFIERS:
            output.append(token)
        elif token in OPERATORS:
            while stack and stack[-1] in OPERATORS and precedence[stack[-1]] >= precedence[token]:
                output.append(stack.pop())
            stack.append(token)
        elif token == '(':
            stack.append(token)
        elif token == ')':
            while stack and stack[-1] != '(':
                output.append(stack.pop())
            stack.pop()
    while stack:
        output.append(stack.pop())
    return output

def encode(tokens, max_len=MAX_LEN):
    ids = [token_to_id[t] for t in tokens] + [EOS_ID]
    return ids + [PAD_ID] * (max_len - len(ids))

def decode_sequence(token_ids, id_to_token, pad_token='PAD', eos_token='EOS'):
    """
    Converts a list of token IDs into a readable string by decoding tokens.
    Stops at the first EOS token if present, and ignores PAD tokens.
    """
    tokens = []
    for token_id in token_ids:
        token = id_to_token.get(token_id, '?')
        if token == eos_token:
            break
        if token != pad_token:
            tokens.append(token)
    return ' '.join(tokens)

def generate_dataset(n,max_depth=MAX_DEPTH):
    X, Y = [], []
    for _ in range(n):
        expr = generate_infix_expression(MAX_DEPTH)
        #expr = expr_gen.generate(max_depth=max_dthep)
        infix = tokenize(expr)
        postfix = infix_to_postfix(infix)
        X.append(encode(infix))
        Y.append(encode(postfix))
    return np.array(X), np.array(Y)

#you might use the shift function for teacher-forcing
def shift_right(seqs):
    shifted = np.zeros_like(seqs)
    shifted[:, 1:] = seqs[:, :-1]
    shifted[:, 0] = SOS_ID
    return shifted

Let us define a simple dataset, and inspect a few samples.

In [4]:
X_train, Y_train = generate_dataset(10000)
decoder_input_train = shift_right(Y_train)

# Dataset
X_val, Y_val = generate_dataset(1000)
decoder_input_val = shift_right(Y_val)

In [5]:
i =  np.random.randint(10000)
print(i)
print("infix : ",decode_sequence(X_train[i],id_to_token))
print("posfix notation: ",decode_sequence(Y_train[i],id_to_token))
print("teacher forcing : ", decode_sequence(decoder_input_train[i],id_to_token))

1765
infix :  ( e - ( e * e ) )
posfix notation:  e e e * -
teacher forcing :  SOS e e e * -


# Constraints
* You may use any architecture (decoder-only, encoder-decoder, or other).

* The maximum number of parameters is 2 million.

* Beam search is not allowed.

* You may adapt the formula generator to your needs, but preserve its core logic—especially the frequency distribution of formulas by depth, as it may significantly influence model performance.

* You may train your model using a pre-generated fixed dataset (e.g., an array) or directly use an on-the-fly generator.




# Evaluation

We shall evaluate a generated item y_pred using "prefix accuracy", the lenght of
the initial prefix of y_pred matching the ground true y_true. This will be divided by the maximum length of y_true and y_pred (up to EOS), so that a perfect match has score 1.

* It's more informative than exact match (which is often 0)

* It’s tighter than edit distance: focuses on generation flow

* Captures where the model starts to make errors



In [6]:
def prefix_accuracy_single(y_true, y_pred, id_to_token, eos_id=EOS_ID, verbose=False):
    t_str = decode_sequence(y_true, id_to_token).split(' EOS')[0]
    p_str = decode_sequence(y_pred, id_to_token).split(' EOS')[0]
    t_tokens = t_str.strip().split()
    p_tokens = p_str.strip().split()
    max_len = max(len(t_tokens), len(p_tokens))

    match_len = sum(x == y for x, y in zip(t_tokens, p_tokens))
    score = match_len / max_len if max_len>0 else 0

    if verbose:
        print("TARGET :", ' '.join(t_tokens))
        print("PREDICT:", ' '.join(p_tokens))
        print(f"PREFIX MATCH: {match_len}/{len(t_tokens)} → {score:.2f}")

    return score

For the exam, evaluate you model on a test set of 20 expressions. Repeat this evaluation 10 times, and return the mean and std for this rounds.

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, Model


# 1. Positional Encoding
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates


def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model
    )
    # apply sin to even indices in the array; cos to odd indices
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]  # shape (1, position, d_model)
    return tf.cast(pos_encoding, dtype=tf.float32)


# 2. Masking
def create_padding_mask(seq):
    # seq: (batch_size, seq_len), padded with PAD_ID
    mask = tf.cast(tf.math.equal(seq, PAD_ID), tf.float32)
    # add extra dimensions to add the padding to the attention logits.
    # shape: (batch_size, 1, 1, seq_len)
    return mask[:, tf.newaxis, tf.newaxis, :]


def create_look_ahead_mask(size):
    # mask the future tokens: shape (size, size)
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    # 1s in upper triangle (excluding diagonal), 0s elsewhere
    return mask  # will broadcast as needed


# 3. Transformer encoder & decoder layers
class EncoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()
        self.mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(dff, activation="relu"), layers.Dense(d_model)]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, x, training=None, mask=None):
        # x: (batch_size, seq_len, d_model)
        attn_output = self.mha(
            query=x, value=x, key=x, attention_mask=mask
        )  # (batch_size, seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)  # (batch_size, seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2


class DecoderLayer(layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()
        self.mha1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.mha2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)

        self.ffn = tf.keras.Sequential(
            [layers.Dense(dff, activation="relu"), layers.Dense(d_model)]
        )

        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)

    def call(
        self, x, enc_output, training=None, look_ahead_mask=None, padding_mask=None
    ):
        # x: (batch_size, target_seq_len, d_model)
        # enc_output: (batch_size, inp_seq_len, d_model)
        # look_ahead_mask: for masking future tokens in decoder self-attention
        # padding_mask: to mask encoder output padding in cross-attention
        attn1 = self.mha1(
            query=x, value=x, key=x, attention_mask=look_ahead_mask
        )  # self-attn
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2 = self.mha2(
            query=out1, value=enc_output, key=enc_output, attention_mask=padding_mask
        )  # cross-attn
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(
            ffn_output + out2
        )  # (batch_size, target_seq_len, d_model)
        return out3


class Encoder(layers.Layer):
    def __init__(
        self,
        num_layers,
        d_model,
        num_heads,
        dff,
        input_vocab_size,
        maximum_position_encoding,
        rate=0.1,
    ):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.enc_layers = [
            EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)
        ]
        self.dropout = layers.Dropout(rate)

    def call(self, x, training=None, mask=None):
        seq_len = tf.shape(x)[1]
        # embedding + positional encoding
        x = self.embedding(x)  # (batch_size, seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training=training, mask=mask)
        return x  # (batch_size, seq_len, d_model)


class Decoder(layers.Layer):
    def __init__(
        self,
        num_layers,
        d_model,
        num_heads,
        dff,
        target_vocab_size,
        maximum_position_encoding,
        rate=0.1,
    ):
        super().__init__()
        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [
            DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)
        ]
        self.dropout = layers.Dropout(rate)

    def call(
        self, x, enc_output, training=None, look_ahead_mask=None, padding_mask=None
    ):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.dec_layers[i](
                x,
                enc_output,
                training=training,
                look_ahead_mask=look_ahead_mask,
                padding_mask=padding_mask,
            )
        # x: (batch_size, target_seq_len, d_model)
        return x


# 4. Full Transformer
def build_transformer(
    input_vocab_size,
    target_vocab_size,
    num_layers=2,
    d_model=128,
    num_heads=4,
    dff=512,
    pe_input=1000,
    pe_target=1000,
    rate=0.1,
):
    # Inputs
    encoder_inputs = layers.Input(
        shape=(None,), name="encoder_inputs"
    )  # (batch_size, inp_seq_len)
    decoder_inputs = layers.Input(
        shape=(None,), name="decoder_inputs"
    )  # (batch_size, tar_seq_len)

    # Masks
    enc_padding_mask = layers.Lambda(create_padding_mask, name="enc_padding_mask")(
        encoder_inputs
    )
    dec_padding_mask = layers.Lambda(create_padding_mask, name="dec_padding_mask")(
        encoder_inputs
    )

    # look-ahead mask for decoder self-attention
    # We'll build look-ahead mask dynamically inside a Lambda:
    def compute_look_ahead_mask(x):
        seq_len = tf.shape(x)[1]
        mask = create_look_ahead_mask(seq_len)
        return mask  # shape (seq_len, seq_len)

    look_ahead_mask = layers.Lambda(compute_look_ahead_mask, name="look_ahead_mask")(
        decoder_inputs
    )
    # also need decoder padding mask for encoder-decoder attention: same as dec_padding_mask above

    # Encoder
    encoder = Encoder(
        num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate
    )
    enc_output = encoder(
        encoder_inputs, mask=enc_padding_mask
    )  # (batch_size, inp_seq_len, d_model)

    # Decoder
    decoder = Decoder(
        num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate
    )
    dec_output = decoder(
        decoder_inputs,
        enc_output,
        look_ahead_mask=look_ahead_mask,
        padding_mask=dec_padding_mask,
    )  # (batch_size, tar_seq_len, d_model)

    # Final linear layer
    final_output = layers.Dense(target_vocab_size, name="final_dense")(
        dec_output
    )  # (batch_size, tar_seq_len, vocab_size)

    return Model(inputs=[encoder_inputs, decoder_inputs], outputs=final_output)


# 5. Loss & Metrics
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction="none"
)


def loss_function(real, pred):
    # real: (batch_size, seq_len), pred: (batch_size, seq_len, vocab_size)
    mask = tf.cast(tf.not_equal(real, PAD_ID), tf.float32)
    loss_ = loss_object(real, pred)
    loss_ *= mask
    return tf.reduce_sum(loss_) / tf.reduce_sum(mask)


def accuracy_function(real, pred):
    # real: (batch_size, seq_len), pred: (batch_size, seq_len, vocab_size)
    pred_ids = tf.argmax(pred, axis=-1, output_type=tf.int32)
    matches = tf.cast(tf.equal(real, pred_ids), tf.float32)
    mask = tf.cast(tf.not_equal(real, PAD_ID), tf.float32)
    matches *= mask
    return tf.reduce_sum(matches) / tf.reduce_sum(mask)


# 6. Hyperparameters (tune as needed)
num_layers = 2
d_model = 128
num_heads = 4
dff = 512
# Estimate max sequence lengths for positional encodings: e.g. based on max infix/postfix lengths
# Suppose you padded/generate sequences up to max_len (you know from your encode logic).
# For safety, set pe_input and pe_target to something >= max length, e.g. 100.
pe_input = 100
pe_target = 100

# Build model
transformer = build_transformer(
    input_vocab_size=VOCAB_SIZE,
    target_vocab_size=VOCAB_SIZE,
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    pe_input=pe_input,
    pe_target=pe_target,
    rate=0.1,
)

transformer.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=loss_function,
    metrics=[accuracy_function],
)


# 7. Prepare data for training
# Example: generate a dataset, pad to fixed length
def prepare_data(batch_size=64, num_samples=10000, max_len=50):
    # generate_dataset returns numpy arrays of shape (n, seq_len) with integer IDs, likely padded/truncated.
    X, Y = generate_dataset(num_samples)
    # Determine max lengths (or specify beforehand)
    # Here assume X and Y are already padded to same length; otherwise we need to pad them:
    # e.g. pad sequences to max_len with PAD_ID on right.
    # For simplicity, assume generate_dataset gives fixed-length arrays; if not, do:
    X = tf.keras.preprocessing.sequence.pad_sequences(
        X, maxlen=max_len, padding="post", value=PAD_ID
    )
    Y = tf.keras.preprocessing.sequence.pad_sequences(
        Y, maxlen=max_len, padding="post", value=PAD_ID
    )
    # Decoder inputs: shift_right(Y)
    dec_inputs = shift_right(Y)
    return X, dec_inputs, Y

2025-06-11 12:10:46.545675: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
# Train
batch_size = 64
epochs = 1
X_train, dec_inp_train, Y_train = prepare_data(
    batch_size=64, num_samples=20000, max_len=50
)
transformer.fit(
    [X_train, dec_inp_train],
    Y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1,
)

  9/282 ━━━━━━━━━━━━━━━━━━━━ 3:19 730ms/step - accuracy_function: 0.1096 - loss: 3.0277

KeyboardInterrupt: 

In [ ]:
def test(no=20,rounds=10):
  rscores =[]
  for i in range(rounds):
    print("round=",i)
    X_test, Y_test = generate_dataset(no)
    scores = []
    for j in range(no):
      encoder_input=X_test[j]
      generated = autoregressive_decode(model, encoder_input)[1:] #remove SOS
      scores.append(prefix_accuracy_single(Y_test[j], generated, id_to_token))
    rscores.append(np.mean(scores))
  return np.mean(rscores),np.std(rscores)

res, std = test(20,10)
print("score=",res,"std=",std)

round= 0


NameError: name 'autoregressive_decode' is not defined

Be sure to evalutate the generator: your model may only take as input the expression in infix format and return its translation to postifix.

If you are usuing an encoder-decoder model, generation must be done autoregressively.

# What to deliver

As usual you are supposed to deliver a single notebook witten in Keras. You are auhtorized to use Keras3 with pytorch as backend if your prefer.

Do no upload a zip file: the submission will be rejected.

The python notebook should have a clear documentation of the training phase, possibly with its history.

You should be able to provide the network paramters upon request. Even better, consider a way to upload them inside your notebook using gdown.